## Build history of commodity COTs

1. Retrieve data from the CFTC website (www.cft.gov/files);
2. Extract data from the most important Commercial and Non Commercial long and short columns;
3. Create "net" columns for each important category;
4. Merge this data with the ETF history data created in the previous steps


In [1]:
import pandas as pd
import numpy as np
import os, sys
import datetime
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import zipfile
import urllib.request
from PIL import Image
def str_to_date(d):
    try:
        dt = datetime.datetime.strptime(str(d),'%Y-%m-%d')
    except:
        return None
    return dt

# Make important folders
TEMP_FOLDER = './temp_folder'
try:
    os.mkdir(TEMP_FOLDER)
except:
    pass
try:
    os.mkdir(f'{TEMP_FOLDER}/cot')
except:
    pass
try:
    os.mkdir(f'{TEMP_FOLDER}/zip')
except:
    pass


___
## First, decide if you want to re-create the ETF and COT data, or just retrieve the previously saved data DataFrames.

In [2]:
CREATE_COT_DATA = True
cot_save_path = './cot_history.csv'

___
## Process CFTC COT Data
___

### Initial processing
1. Download zip files from www.cft.gov/files;
2. Unip the files using the zipfile package;
3. Read each csv (usually named Annual.TXT), and merge them into the df_cot DataFrame.


In [3]:
if CREATE_COT_DATA:
    last_year = datetime.datetime.now().year
    year_list = np.linspace(2000,last_year,last_year-2000+1)
    zip_download_folder = f'TEMP_FOLDER/zip'
    df_cot_temp = None
    df_cot = None
    for y in year_list:
        yint = int(y)
        url = f"https://www.cftc.gov/files/dea/history/deacot{yint}.zip"
        path_to_zip_file = f'{zip_download_folder}/dea_fut_xls_{y}.zip'
        if not os.path.isfile(path_to_zip_file) or y >= last_year:
            print(f'retrieving cot zip file from {url}')
            try:
                urllib.request.urlretrieve(url, path_to_zip_file)    
            except:
                import time
                time.sleep(1)
                urllib.request.urlretrieve(url, path_to_zip_file)    
        zip_ref = zipfile.ZipFile(path_to_zip_file, 'r')
        zip_ref.extractall(zip_download_folder)
        zip_ref.close()
        df_cot_temp = pd.read_csv(f'{zip_download_folder}/Annual.TXT')
        if df_cot is None:
            df_cot = df_cot_temp.copy()
        else:
            df_cot = df_cot.append(df_cot_temp,ignore_index=True)
            df_cot.index = list(range(len(df_cot)))
        print(f'processed cot csv file from {url}. Length = {len(df_cot_temp)}')
        
    df_cot.head()

processed cot csv file from https://www.cftc.gov/files/dea/history/deacot2000.zip. Length = 3261
processed cot csv file from https://www.cftc.gov/files/dea/history/deacot2001.zip. Length = 2782
processed cot csv file from https://www.cftc.gov/files/dea/history/deacot2002.zip. Length = 2854
processed cot csv file from https://www.cftc.gov/files/dea/history/deacot2003.zip. Length = 3135
processed cot csv file from https://www.cftc.gov/files/dea/history/deacot2004.zip. Length = 3465
processed cot csv file from https://www.cftc.gov/files/dea/history/deacot2005.zip. Length = 3909
processed cot csv file from https://www.cftc.gov/files/dea/history/deacot2006.zip. Length = 4926
processed cot csv file from https://www.cftc.gov/files/dea/history/deacot2007.zip. Length = 4926
processed cot csv file from https://www.cftc.gov/files/dea/history/deacot2008.zip. Length = 5631
processed cot csv file from https://www.cftc.gov/files/dea/history/deacot2009.zip. Length = 6570
processed cot csv file from ht

/Users/bperlman1/Virtualenvs3/pyliverisk/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning:

Columns (3,37,38,39,40,41,42,43,44,45,46,126) have mixed types. Specify dtype option on import or set low_memory=False.



processed cot csv file from https://www.cftc.gov/files/dea/history/deacot2017.zip. Length = 12314
processed cot csv file from https://www.cftc.gov/files/dea/history/deacot2018.zip. Length = 13410
retrieving cot zip file from https://www.cftc.gov/files/dea/history/deacot2019.zip
processed cot csv file from https://www.cftc.gov/files/dea/history/deacot2019.zip. Length = 2245


___
### Make column names easier to process, make main date field a datetime object, and sort the DataFrame
___

In [4]:
if CREATE_COT_DATA:
    col_rename_dict = {c:c.replace(' ','_').replace('-','_').replace('(','').replace(')','') for c in df_cot.columns.values}
    df_cot2 = df_cot.rename(columns=col_rename_dict)
    df_cot2.As_of_Date_in_Form_YYYY_MM_DD = df_cot2.As_of_Date_in_Form_YYYY_MM_DD.apply(str_to_date)
    df_cot2 = df_cot2.sort_values(['Market_and_Exchange_Names','As_of_Date_in_Form_YYYY_MM_DD'])
    df_cot2.columns.values
    df_cot2.to_csv(cot_save_path,index=False)

___
### Show important columns for a specific  commodity
___

In [10]:
df_cot2 = pd.read_csv(cot_save_path)
df_cot2.As_of_Date_in_Form_YYYY_MM_DD = df_cot2.As_of_Date_in_Form_YYYY_MM_DD.apply(str_to_date)
commod = 'CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE EXCHANGE'
# commod = 'GOLD'
cot_beg_date = datetime.datetime.now() - datetime.timedelta(2000)
df_commod = df_cot2[df_cot2.Market_and_Exchange_Names.str.contains(commod)][df_cot2.As_of_Date_in_Form_YYYY_MM_DD>=cot_beg_date]
basic_cols = ['Market_and_Exchange_Names','As_of_Date_in_Form_YYYY_MM_DD','Open_Interest_All']
long_cols = ['Market_and_Exchange_Names','As_of_Date_in_Form_YYYY_MM_DD',
            'Noncommercial_Positions_Long_All','Commercial_Positions_Long_All',
            'Nonreportable_Positions_Long_All','Traders_Commercial_Long_All',
             'Traders_Noncommercial_Long_All','Traders_Total_Reportable_Long_All']
short_cols = ['Market_and_Exchange_Names','As_of_Date_in_Form_YYYY_MM_DD',
            'Noncommercial_Positions_Short_All','Commercial_Positions_Short_All',
            'Nonreportable_Positions_Short_All','Total_Reportable_Positions_Short_All',
            'Traders_Commercial_Short_All','Traders_Noncommercial_Short_All',
            'Traders_Total_Reportable_Short_All']
df_commod_basic = df_commod[basic_cols]
df_commod_long = df_commod[long_cols]
df_commod_short = df_commod[short_cols]


/Users/bperlman1/Virtualenvs3/pyliverisk/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



### Show basic open interest info

In [11]:
df_commod_basic.sort_values('As_of_Date_in_Form_YYYY_MM_DD',ascending=False).head()

,Market_and_Exchange_Names,As_of_Date_in_Form_YYYY_MM_DD,Open_Interest_All
26164,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-03-05,2067851
26163,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-02-26,2025884
26162,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-02-19,2019218
26161,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-02-12,2077398
26160,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-02-05,2057061


### Show important "long" position info

In [12]:
df_commod_long.sort_values('As_of_Date_in_Form_YYYY_MM_DD',ascending=False).head()

,Market_and_Exchange_Names,As_of_Date_in_Form_YYYY_MM_DD,Noncommercial_Positions_Long_All,Commercial_Positions_Long_All,Nonreportable_Positions_Long_All,Traders_Commercial_Long_All,Traders_Noncommercial_Long_All,Traders_Total_Reportable_Long_All
26164,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-03-05,497656,811089,81756,102,120,320
26163,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-02-26,486235,822211,79354,95,120,311
26162,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-02-19,479000,809084,78386,100,117,309
26161,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-02-12,472181,841465,79391,105,114,323
26160,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-02-05,491870,816098,81434,101,120,322


### Show important "short" position info

In [13]:
df_commod_short.sort_values('As_of_Date_in_Form_YYYY_MM_DD',ascending=False).head()

,Market_and_Exchange_Names,As_of_Date_in_Form_YYYY_MM_DD,Noncommercial_Positions_Short_All,Commercial_Positions_Short_All,Nonreportable_Positions_Short_All,Total_Reportable_Positions_Short_All,Traders_Commercial_Short_All,Traders_Noncommercial_Short_All,Traders_Total_Reportable_Short_All
26164,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-03-05,148855,1168664,72982,1994869,107,96,296
26163,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-02-26,158564,1159831,69405,1956479,107,98,298
26162,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-02-19,165033,1131507,69930,1949288,109,101,291
26161,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-02-12,183967,1140393,68677,2008721,111,112,305
26160,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-02-05,170261,1148220,70921,1986140,106,111,299


### Show important "net" position info

In [14]:
def non_comm_net(r):
    return float(r.Noncommercial_Positions_Long_All) - float(r.Noncommercial_Positions_Short_All)
def comm_net(r):
    return float(r.Commercial_Positions_Long_All) - float(r.Commercial_Positions_Short_All)
def non_report_net(r):
    return float(r.Nonreportable_Positions_Long_All) - float(r.Nonreportable_Positions_Short_All)
def traders_comm_net(r):
    return float(r.Traders_Commercial_Long_All) - float(r.Traders_Commercial_Short_All)
def traders_noncomm_net(r):
    return float(r.Traders_Noncommercial_Long_All) - float(r.Traders_Noncommercial_Short_All)

df_commod_net = df_commod_long.merge(df_commod_short,how='inner',on=['Market_and_Exchange_Names','As_of_Date_in_Form_YYYY_MM_DD'])
df_commod_net['Noncommercial_Positions_Net_All'] = df_commod_net.apply(non_comm_net,axis=1)
df_commod_net['Commercial_Positions_Net_All'] = df_commod_net.apply(comm_net,axis=1)
df_commod_net['Nonreportable_Positions_Net_All'] = df_commod_net.apply(non_report_net,axis=1)
df_commod_net['Traders_Commercial_Net_All'] = df_commod_net.apply(traders_comm_net,axis=1)
df_commod_net['Traders_Noncommercial_Net_All'] = df_commod_net.apply(traders_noncomm_net,axis=1)
net_cols = ['Market_and_Exchange_Names','As_of_Date_in_Form_YYYY_MM_DD','Noncommercial_Positions_Net_All','Commercial_Positions_Net_All','Nonreportable_Positions_Net_All','Traders_Commercial_Net_All','Traders_Noncommercial_Net_All']
df_commod_net = df_commod_net[net_cols]
df_commod_net.sort_values('As_of_Date_in_Form_YYYY_MM_DD',ascending=False).head()


,Market_and_Exchange_Names,As_of_Date_in_Form_YYYY_MM_DD,Noncommercial_Positions_Net_All,Commercial_Positions_Net_All,Nonreportable_Positions_Net_All,Traders_Commercial_Net_All,Traders_Noncommercial_Net_All
217,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-03-05,348801.0,-357575.0,8774.0,-5.0,24.0
216,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-02-26,327671.0,-337620.0,9949.0,-12.0,22.0
215,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-02-19,313967.0,-322423.0,8456.0,-9.0,16.0
214,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-02-12,288214.0,-298928.0,10714.0,-6.0,2.0
213,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-02-05,321609.0,-332122.0,10513.0,-5.0,9.0


## END